In [ ]:
from src.dataset import CustomDataset
!git clone https://github.com/inttx/DLAM_SealedSurfaces.git
%cd DLAM_SealedSurfaces

In [ ]:
!pip install -r requirements.txt
%cd src

In [ ]:
from settings import *
from dataset import CustomDataset, PotsdamDataset, HessigheimDataset
from models import get_trained_custom_resnet18, get_trained_deeplabv3_resnet101, get_trained_segformer_model
from evaluate import evaluate_segmentation

In [ ]:
try:
    import os
    import shutil
    from google.colab import drive
    MOUNTPOINT = '/content/drive/'
    drive.mount(MOUNTPOINT)

    IMAGE_PATH = os.path.join(MOUNTPOINT, 'MyDrive', 'DLAM', '2_Ortho_RGB') # TODO adjust to your path
    LABEL_PATH = os.path.join(MOUNTPOINT, 'MyDrive', 'DLAM', '5_Labels_all') # TODO adjust to your path

    EVAL_PATH = MOUNTPOINT + 'MyDrive/DLAM/eval/'
except:
    IMAGE_PATH = '../data/2_Ortho_RGB' # TODO adjust to your path
    LABEL_PATH = '../data/5_Labels_all' # TODO adjust to your path
    EVAL_PATH = '../eval/'
os.makedirs(EVAL_PATH, exist_ok=True)

In [ ]:
# Hyperparameters
patch_size = 250
stride = 250
batch_size = 8
classification_mode = "single" # or "combined"

datasetType: type[CustomDataset] = PotsdamDataset # or HessigheimDataset
num_classes = datasetType.get_num_classes(classification_mode)
class_names = datasetType.class_names_single() if classification_mode == "single" else datasetType.class_names_combined()

# Train resnet18

In [ ]:
dataset = datasetType(IMAGE_PATH, LABEL_PATH, patch_size=patch_size, stride=stride, device=DEVICE)
train_loader, val_loader, test_loader = datasetType.get_data_loaders(IMAGE_PATH, LABEL_PATH,patch_size,
                     stride,
                     dist=(0.8, 0.1, 0.1),
                     batch_size=batch_size,
                     device=DEVICE,
                     mode=classification_mode,
                     pin_memory=True,
                     num_workers=2,
                     seed=42)

model = get_trained_custom_resnet18(model_path='../models/resnet18.pth', patch_size=patch_size, num_classes=num_classes, device=DEVICE)
evaluate_segmentation(model=model, data_loader=test_loader, device=DEVICE, model_type='ResNet18', class_names=class_names, patch_size=patch_size, eval_path=EVAL_PATH + 'resnet18.svg')

# Train DeepLabV3 resnet101 baseline

In [ ]:
dataset = datasetType(IMAGE_PATH, LABEL_PATH, patch_size=patch_size, stride=stride, device=DEVICE)
train_loader, val_loader, test_loader = datasetType.get_data_loaders(IMAGE_PATH, LABEL_PATH,patch_size,
                     stride,
                     dist=(0.8, 0.1, 0.1),
                     batch_size=batch_size,
                     device=DEVICE,
                     mode=classification_mode,
                     pin_memory=True,
                     num_workers=2,
                     seed=42)

model = get_trained_deeplabv3_resnet101(model_path='../models/deeplabv3_resnet101.pth', num_classes=num_classes, device=DEVICE)

evaluate_segmentation(model=model, data_loader=test_loader, device=DEVICE, model_type='DeepLabV3', class_names=class_names, patch_size=patch_size, eval_path=EVAL_PATH + 'deeplabv3_resnet101.svg')

# Train SegFormer

In [ ]:
dataset = datasetType(IMAGE_PATH, LABEL_PATH, patch_size=patch_size, stride=stride, device=DEVICE)
train_loader, val_loader, test_loader = datasetType.get_data_loaders(IMAGE_PATH, LABEL_PATH,patch_size,
                     stride,
                     dist=(0.8, 0.1, 0.1),
                     batch_size=batch_size,
                     device=DEVICE,
                     mode=classification_mode,
                     pin_memory=True,
                     num_workers=2,
                     seed=42)

model = get_trained_segformer_model('../models/segformer.pth', num_classes=num_classes, device=DEVICE)

evaluate_segmentation(model=model, data_loader=test_loader, device=DEVICE, model_type='SegFormer', class_names=class_names, patch_size=patch_size, eval_path=EVAL_PATH + 'segformer.svg')